In [ ]:
# nltk.download('stopwords')
# nltk.download('wordnet')

In [ ]:
# !python -m spacy download en_core_web_sm
# !python -m spacy download en
# !python -m spacy download en_core_web_md

In [1]:
import pandas as pd
import os
from bs4 import BeautifulSoup
import requests
import re
import nltk
import textcleaner
import pickle
import spacy
import jieba

from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize, sent_tokenize
from textblob import TextBlob
import textcleaner as tc
from nltk.stem import WordNetLemmatizer 
import re
import spacy

## Initial data cleaning

In [2]:
pickling_out = open('/Users/greenapple/project4/data/interim/data_110619_seattle_0to1k.pkl', 'rb')
big_data = pickle.load(pickling_out)

In [6]:
# Remove duplicates and data rows that were inserted by indeed
big_data.drop_duplicates(inplace=True)
big_data.drop(big_data.loc[big_data.company_name == 'Seen by Indeed'].index, inplace=True)
big_data.reset_index(drop=True, inplace=True)
big_data.shape

(956, 6)

In [8]:
train_data = big_data[:100]

In [9]:
pickling_out = open('/Users/greenapple/project4/data/interim/data_110619_seattle_neg.pkl', 'rb')
data_neg = pickle.load(pickling_out)

In [10]:
neg_set = data_neg[10:20]

In [12]:
# Make test job data set
test_data_pos = big_data[:5]
test_data_neg  = data_neg[5:10]
test_data = pd.concat([test_data_pos, test_data_neg])
test_data.shape

(10, 6)

In [15]:
neg_set.drop_duplicates(inplace=True)
neg_set.drop(neg_set.loc[neg_set.company_name == 'Seen by Indeed'].index, inplace=True)
neg_set.reset_index(drop=True, inplace=True)
neg_set.shape

/Users/greenapple/anaconda3/envs/project4e/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/greenapple/anaconda3/envs/project4e/lib/python3.7/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


(10, 6)

In [16]:
# Collect data for the rows replaced by indeed
# replaced_data_0to1k = data_110619_seattle_set1.loc[data_110619_seattle_set1.company_name=='Seen by Indeed']

In [17]:
# Load metis resumes
pickling_out = open('/Users/greenapple/project4/data/resumes/resumes_110719', 'rb')
resumes_l = pickle.load(pickling_out)

In [18]:
resumes = pd.DataFrame(resumes_l, columns = ['description'])

In [19]:
resumes

,description
0,"ROB\nJOHNS,\nPH.D\n\nDATA SCIENTIST\n& Robjohn..."
1,JEREMY LEHNER\n\nSUMMARY\n\nDATA SCIENTIST | D...
2,DAVID ALEXANDER LOU\n\nDATA SCIENTIST | ECONOM...
3,"FATIMA LOUMAINI, CSM CCBA DATA SCIENTIST\n\n ..."
4,Aisulu Omar\n\nLINKS\n\nPortfolio\nhttps://www...


## Models

### Functions

In [28]:
# Pre-process text
def pre_process(text):
    text_sub = re.sub('[/.\n:-]', ' ', text)  # Remove "/" and "."
    text_sub = re.sub('[,\(\)]', '', text_sub).lower()  # Remove "," and capital letters
    text_sub = re.sub('[^a-z\s]', '', text_sub) # Keep alphanumer characters only
#     re.sub('[%s]' % re.escape(string.punctuation), ' ', my_text)

    return text_sub

In [29]:
# Tokenize text
def tokenize_lemma(text):
        
    # processer = spacy.load('en')
    tokenizer = spacy.load('en_core_web_sm')

    text_obj = tokenizer(text)
    text_str = ' '.join([token.lemma_ for token in text_obj if not token.is_stop])
    return text_str

In [30]:
# Vectorize
def tf_idf(text1, text2):
    tfidf = TfidfVectorizer(ngram_range=(1, 2),               # vectorizer
                        stop_words='english', 
                        token_pattern="\\b[a-z][a-z]+\\b")
    X_tfidf = tfidf.fit_transform(text1).toarray()            # vectorize text1
    Z_tfidf = tfidf.transform(text2).toarray()                # vectorize text1
    
    
#     print('text1{}'.format(pd.DataFrame(X_tfidf, columns=tfidf.get_feature_names()).head(5)))
#     print('text2{}'.format(pd.DataFrame(Z_tfidf, columns=tfidf.get_feature_names()).head(5)))
    
    return X_tfidf, Z_tfidf

In [31]:
# Reduce dimensionality:
def lsa(X, Z):
    lsa = TruncatedSVD(4)
    X_lsa = lsa.fit_transform(X)
    Z_lsa = lsa.transform(Z)
  
    
    return X_lsa, Z_lsa

In [32]:
def similarity(X_red, Z_red):
    score = cosine_similarity(X_red, Z_red).round(2)
    
    column_labels = ['Resume'+str(i) for i in range(1, R_lsa.shape[0]+1)]
    row_labels = ['Job'+str(i) for i in range(1, Z_lsa.shape[0]+1)]
   
    similarity_df = pd.DataFrame(score, columns=column_labels, index=row_labels)
#     print(similarity_df)
        
    return similarity_df

In [33]:
def similarity(X_red, Z_red):
    score = cosine_similarity(X_red, Z_red).round(2)
    
    column_labels = ['Resume'+str(i) for i in range(1, Z_red.shape[0]+1)]
    row_labels = ['Job'+str(i) for i in range(1, X_red.shape[0]+1)]
   
    similarity_df = pd.DataFrame(score, columns=column_labels, index=row_labels)
#     print(similarity_df)
        
    return similarity_df

### Toy example

In [34]:
# Clean, lemma, tf-idf, lsa, cosine similarity

train_data['text'] = train_data.description.apply(pre_process) # clean
train_data['lemma'] = train_data.text.apply(tokenize_lemma) # tokenize and lemmatize

test_data_pos['text'] = test_data_pos.description.apply(pre_process)  # clean
test_data_pos['lemma'] = test_data_pos.text.apply(tokenize_lemma) # tokenize and lemmatize

resumes['text'] = resumes.description.apply(pre_process)  # clean
resumes['lemma'] = resumes.text.apply(tokenize_lemma) # tokenize and lemmatize

Tr_tfidf, R_tfidf = tf_idf(train_data.text, resumes.text) # vectorize
Tr_tfidf, Ts_tfidf = tf_idf(train_data.text, test_data_pos.text) # vectorize

Tr_lsa, R_lsa = lsa(Tr_tfidf, R_tfidf)  # reduce dimensionality
Tr_lsa, Ts_lsa = lsa(Tr_tfidf, Ts_tfidf)  # reduce dimensionality

score = similarity(Ts_lsa, R_lsa) # cosine similarity

/Users/greenapple/anaconda3/envs/project4e/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/greenapple/anaconda3/envs/project4e/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/greenapple/anaconda3/envs/project4e/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyW

In [35]:
score

,Resume1,Resume2,Resume3,Resume4,Resume5
Job1,0.74,0.81,0.66,0.69,0.79
Job2,0.81,0.87,0.74,0.76,0.85
Job3,0.73,0.70,0.83,0.55,0.72
Job4,0.74,0.81,0.66,0.69,0.79
Job5,0.95,0.97,0.89,0.93,0.96


In [37]:
train_data

,company_name,description,job_title,link,location,salary,text,lemma
0,red violet,Data ScientistDowntown SeattleWe’re looking fo...,Data Scientist,https://www.indeed.com//pagead/clk?mo=r&ad=-6N...,"Seattle, WA",NaN,data scientistdowntown seattlewere looking for...,datum scientistdowntown seattlewere look data ...
1,eXcell,As part of a leading IT managed services speci...,Database Engineer / Data Scientist,https://www.indeed.com//pagead/clk?mo=r&ad=-6N...,"Redmond, WA 98052",NaN,as part of a leading it managed services speci...,lead manage service specialist associate wor...
2,Porch,"Porch is looking for an accomplished, hands-on...",Lead Data Scientist,https://www.indeed.com//pagead/clk?mo=r&ad=-6N...,"Seattle, WA",NaN,porch is looking for an accomplished hands on ...,porch look accomplished hand datum scientist l...
3,red violet,Data ScientistDowntown SeattleWe’re looking fo...,Data Scientist,https://www.indeed.com//pagead/clk?mo=r&ad=-6N...,"Seattle, WA",NaN,data scientistdowntown seattlewere looking for...,datum scientistdowntown seattlewere look data ...
4,Cequint,The Splunk Analyst is responsible for supporti...,Data Analyst,https://www.indeed.com//rc/clk?jk=9aa6db5a5410...,"Seattle, WA 98104",NaN,the splunk analyst is responsible for supporti...,splunk analyst responsible support effort datu...
...,...,...,...,...,...,...,...,...
95,TransWest,TransWest - Driving BusinessWe're looking for ...,Program Manager- Data Analyst,https://www.indeed.com//pagead/clk?mo=r&ad=-6N...,Data Analyst,"$62,141 - $70,468 a year",transwest driving businesswere looking for a...,transwest drive businesswere look experienc...
96,University of Washington,"Notes: As a UW employee, you will enjoy genero...",SENIOR DATA SCIENTIST,https://www.indeed.com//rc/clk?jk=5315b0b82919...,"Seattle, WA","$62,141 - $70,468 a year",notes as a uw employee you will enjoy generou...,note uw employee enjoy generous benefit work...
97,Tesla,We are looking for a Sr Data Scientist to help...,Data Scientist/Machine Learning Engineer,https://www.indeed.com//pagead/clk?mo=r&ad=-6N...,"Seattle, WA","$62,141 - $70,468 a year",we are looking for a sr data scientist to help...,look sr data scientist help develop digital ma...
98,Association of Washington Cities,Association of Washington Cities (AWC) is recr...,Applications and Data Analyst,https://www.indeed.com//pagead/clk?mo=r&ad=-6N...,"Olympia, WA 98501","$62,141 - $70,468 a year",association of washington cities awc is recrui...,association washington cities awc recruit time...
